### Parameters & Imports

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 320, 240
train_data_dir = 'datasets/UCF11/frames/train'
validation_data_dir = 'datasets/UCF11/frames/test'
nb_train_samples = 7365
nb_validation_samples = 1825 
batch_size = 64
epochs = 50
nb_class = 11

### Define Model

In [ ]:
base_model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

x = base_model.output
predictions = GlobalAveragePooling2D()(x)

model = Model(inputs=base_model.input, outputs=predictions)

### Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, 
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, 
    class_mode='categorical'
)

### Deploy Model

In [ ]:
train_files = train_generator.filenames
test_files = validation_generator.filenames

train_features = model.predict_generator(
    frame_generator,
    steps=nb_train_samples // batch_size
)

test_features = model.predict_generator(
    validation_generator,
    steps=nb_validation_samples // batch_size
)

### Save Results

In [ ]:
import os
import numpy as np

for file, feature in zip(train_files, train_features):
    path = os.path.join(features_data_dir + '/train', os.path.splitext(file)[0] ) + '.npy'
    if not os.path.isdir( os.path.dirname(path) ):
        os.makedirs( os.path.dirname(path) )
        np.save(path, feature)
        
for file, feature in zip(test_files, test_features):
    path = os.path.join(features_data_dir + '/test', os.path.splitext(file)[0] ) + '.npy'
    if not os.path.isdir( os.path.dirname(path) ):
        os.makedirs( os.path.dirname(path) )
        np.save(path, feature)